# **第2回 演習Ⅱ 課題2, 3**

課題2：コードを実行してください。

課題3：他の画像を使って推論を行ってください。

※上部にある「ドライブにコピー」で自分のドライブにコピーしてから編集・実行してください。

In [ ]:
# ライブラリのインポート
import os
import random
import numpy as np
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [ ]:
# Google DriveをマウントしてDrive内のファイルにアクセスできるようにする
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# シード値の固定
# シードを固定すると毎回同じ結果が得られる（再現性の確保）
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
# データの前処理の方法を定義
transform = transforms.Compose([
	transforms.ToTensor() # データをテンソルに変換
])

In [ ]:
# 畳み込みニューラルネットワークの定義
class CNN(nn.Module):
  def __init__(self):
    '''
    畳み込みニューラルネットワークの構造を定義
    '''
    super(CNN, self).__init__()

    # 畳み込み層1
    self.conv1 = nn.Conv2d(1, 8, 3)  # 28x28x1 -> 26x26x8
    # 畳み込み層2
    self.conv2 = nn.Conv2d(8, 16, 3) # 26x26x8 -> 24x24x16
    # 最大プーリング層
    self.pool = nn.MaxPool2d(2, 2)   # 24x24x16 -> 12x12x16
    # 全結合層
    self.fc1 = nn.Linear(12 * 12 * 16, 128) # 12x12x16 -> 128
    self.fc2 = nn.Linear(128, 64)           # 128 -> 64
    self.fc3 = nn.Linear(64, 10)            # 64 -> 10

  def forward(self, x):
    '''
    入力から出力までの流れを定義
    '''
    x = F.relu(self.conv1(x))    # 畳み込み層1の計算 + 活性化関数ReLUの適用
    x = F.relu(self.conv2(x))    # 畳み込み層2の計算 + 活性化関数ReLUの適用
    x = self.pool(x)             # 最大プーリング

    x = x.view(-1, 12 * 12 * 16) # 特徴マップを1次元のベクトルに変換

    x = F.relu(self.fc1(x))      # 全結合層1の計算 + 活性化関数ReLUの適用
    x = F.relu(self.fc2(x))      # 全結合層2の計算 + 活性化関数ReLUの適用
    x = self.fc3(x)              # 出力層の計算
    return x

In [ ]:
# 学習済みのモデルの読み込み
model = CNN()
model_path = "/content/drive/MyDrive/jts_ai_seminar_2/model/cnn.pt"
model.load_state_dict(torch.load(model_path, weights_only=True))
model.eval() # モデルを評価モードにする

# 推論する画像の読み込み
img_path = "/content/drive/MyDrive/jts_ai_seminar_2/images/image_A.jpg"
img = Image.open(img_path)

# 画像をテンソルに変換
img_tensor = transform(img)
img_tensor = img_tensor.unsqueeze(0)

# モデルに画像データを入力
output = model(img_tensor)

# 出力に「ソフトマックス関数」を適用
pred_proba = F.softmax(output, dim=1) ##### 課題4 #####

# 一番高い確率を持つクラス（数字）を取得
pred_number = int(pred_proba.argmax(dim=1, keepdim=True)[0][0])

# 結果を表示
print("数字\t確率")
for i in range(10):
  print("{}\t{}".format(i, round(float(pred_proba[0][i]), 3)))

print("\n【推論結果】")
# 予測した結果を表示（どの数字に分類されたか）
print("画像「{}」は「{}」です。".format(img_path, pred_number))

# 画像の表示
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.show()